In [103]:
import requests
from pandas import json_normalize
import psycopg2 as pg
import pandas.io.sql as psql
from sklearn import preprocessing

In [104]:
le = preprocessing.LabelEncoder()

In [105]:
url = 'http://bootcamp-ml.westus2.cloudapp.azure.com:5000/data'

response = requests.get(url)
dictr = response.json()
df = json_normalize(dictr)
#print(df)

In [106]:
df.columns

Index(['age', 'radiologists_assessment', 'cancer_diagnosis',
       'comparison_mammogram', 'patients_birads', 'family_history',
       'hormone_therapy', 'indicator', 'breast_biopsy', 'mammogram',
       'cancer_type', 'body_mass', 'patients_id'],
      dtype='object')

In [107]:
dataframeWF = df

In [108]:
dataframeWF['breast_biopsy'][dataframeWF["breast_biopsy"]=='Missing'] = 'No'
dataframeWF['comparison_mammogram'][dataframeWF["comparison_mammogram"]=='Missing'] = 'No'
dataframeWF['family_history'][dataframeWF["family_history"]=='Missing'] = 'No'
dataframeWF['hormone_therapy'][dataframeWF["hormone_therapy"]=='Missing'] = 'No'

dataframeWF["breast_biopsy"].unique()


<ipython-input-108-d1f3fe4b05f9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframeWF['breast_biopsy'][dataframeWF["breast_biopsy"]=='Missing'] = 'No'
<ipython-input-108-d1f3fe4b05f9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframeWF['comparison_mammogram'][dataframeWF["comparison_mammogram"]=='Missing'] = 'No'
<ipython-input-108-d1f3fe4b05f9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframeWF['family_histo

array(['No', 'Yes'], dtype=object)

In [109]:
le.fit(dataframeWF["patients_birads"].unique())
dataframeWF["patients_birads"] = le.transform(dataframeWF["patients_birads"])


In [110]:
le.classes_

array(['Almost entirely fatty', 'Extremely dense',
       'Heterogeneously dense', 'Scattered fibroglandular densities'],
      dtype=object)

In [111]:
dataframeWF["breast_biopsy"].unique()

array(['No', 'Yes'], dtype=object)

In [112]:
le.fit(dataframeWF["breast_biopsy"].unique())
dataframeWF["breast_biopsy"] = le.transform(dataframeWF["breast_biopsy"])


In [113]:
le.classes_

array(['No', 'Yes'], dtype=object)

In [114]:
le.fit(dataframeWF["comparison_mammogram"].unique())
dataframeWF["comparison_mammogram"] = le.transform(dataframeWF["comparison_mammogram"])


In [115]:
le.classes_

array(['No', 'Yes'], dtype=object)

In [116]:
le.fit(dataframeWF["family_history"].unique())
dataframeWF["family_history"] = le.transform(dataframeWF["family_history"])


In [117]:
le.classes_

array(['No', 'Yes'], dtype=object)

In [118]:
le.fit(dataframeWF["hormone_therapy"].unique())
dataframeWF["hormone_therapy"] = le.transform(dataframeWF["hormone_therapy"])


In [119]:
le.classes_

array(['No', 'Yes'], dtype=object)

In [120]:
le.fit(dataframeWF["cancer_diagnosis"].unique())
dataframeWF["cancer_diagnosis"] = le.transform(dataframeWF["cancer_diagnosis"])


In [121]:
le.classes_

array([False,  True])

In [122]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataframeWF[["age","patients_birads","breast_biopsy","comparison_mammogram","family_history","hormone_therapy"]], dataframeWF["cancer_diagnosis"], random_state=42)


In [123]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train,y_train)
rf.score(X_test,y_test)

0.9931

In [124]:
importances = rf.feature_importances_
importances

array([0.63688351, 0.15819435, 0.04833243, 0.0375217 , 0.05588783,
       0.06318018])

In [125]:
sorted(zip(rf.feature_importances_, ["age","patients_birads","breast_biopsy","comparison_mammogram","family_history","hormone_therapy"]), reverse=True)


[(0.6368835100902205, 'age'),
 (0.15819435242452887, 'patients_birads'),
 (0.06318018004194115, 'hormone_therapy'),
 (0.05588782953076964, 'family_history'),
 (0.04833243112267858, 'breast_biopsy'),
 (0.03752169678986128, 'comparison_mammogram')]

In [126]:
import pandas as pd
prediction=pd.DataFrame(rf.predict(X_test))

In [127]:
test = pd.DataFrame(X_test)
test.shape

(10000, 6)

In [128]:
prediction.shape

(10000, 1)

In [129]:
prediction[0].unique()

array([0, 1], dtype=int64)

In [130]:
import pickle
import numpy as np

In [131]:
pickle.dump(rf, open("model.sav", 'wb'))

In [132]:

loaded_model = pickle.load(open("model.sav", 'rb'))
predictors = [84,3,1,2,1,1]
#predictors = predictors.reshape(-1,1)
result = loaded_model.predict([[62,2,2,1,1,2]])
result

array([0], dtype=int64)

In [133]:
X_test

,age,patients_birads,breast_biopsy,comparison_mammogram,family_history,hormone_therapy
7516,84,3,0,1,0,0
13705,67,0,0,1,0,0
21102,60,3,0,1,1,0
10031,67,2,0,1,1,0
18671,68,0,0,1,0,0
...,...,...,...,...,...,...
39643,65,3,0,1,1,0
31431,82,2,0,1,0,0
3036,65,3,0,1,0,0
5604,65,1,1,1,0,0


In [134]:
for index, row in X_test.iterrows():
    #print(f"Age:{row['age']},Patiens_birads:{row['patients_birads']},breast_biopsy:{row['patients_birads']},comparison_mammogram:{row['comparison_mammogram']},family_history:{row['family_history']},hormone_theraoy:{row['hormone_therapy']}")
    predictors = [row['age'],row['patients_birads'],row['patients_birads'],row['comparison_mammogram'],row['family_history'],row['hormone_therapy']]
    result = loaded_model.predict([predictors])
    if (result[0]==1):
        print(predictors)

[80, 1, 1, 0, 0, 0]
[68, 1, 1, 1, 0, 1]
